### Zero-Shot Notebook (Benchmarking Pipeline):

The following notebook is used to answer Research Question #1 (RQ1) in the SLM Threat Query Analysis Project.

### Step #1: Import Packages:

Import all the packages that are needed to run the notebook. To install packages, you will need to run the following command in either your Terminal or in any of the cells:

```!pip install -r ../requirements.txt```

In [ ]:
# Run this cell:
import pandas as pd
from kql_benchmarking_pipeline import KQLBenchmarkPipeline

import re
import time
import os
import tiktoken
import json
import yaml

from ZeroShot import *

with open("../config.yaml", "r") as f:
    token_config = yaml.safe_load(f)

### Step #2: Import Evaluation Dataset:

Read the Evaluation Dataset into a DataFrame, and create a new DataFrame to store the results.

In [ ]:
# Run this cell:

eval_df = pd.read_json(path_or_buf='../NL2KQL_Remakes/data/evaluation/Defender_Evaluation.jsonl', lines=True)
df = pd.DataFrame(columns = ['NLQ', 'KQL'])

### Step #3: Specify the Model:

Next you will need to specify which type of model that you will use. There's multiple options that you can specify, below are some examples:

- HuggingFace-Based Models: 
    - "google/gemma-3-1b-it" (Google's Gemma-3-1B-IT model)
    - "google/gemma-3-4b-it" (Google's Gemma-3-4B-IT model)
    - "microsoft/phi-4" (Microsoft's Phi-4 model)
    - "microsoft/phi-4-mini-instruct" (Microsoft's Phi-4-Mini-Instruct model)

- GenAI-Based Models:
    - "gemini-2.0-flash" (Google's Gemini 2.0 Flash model)

- OpenAI-Based Models:
    - "gpt-4o" (OpenAI's GPT-4o model)
    - "gpt-5" (OpenAI's GPT-5 model)

You can add additional models that are not present in these lists, but you must ensure that they are in the correct specified format. For example, if you plan to load a model from Huggingface, you must set your ```mode``` variable to ```huggingface```. If you wish to load a model from OpenAI, you must set the ```mode``` variable to ```openai```, and if you wish to load a model using the Google GenAI client, you must specify the ```mode``` variable to ```genai-client```. In addition, fill in the variable below with the proper model to use. 

If you plan to use a different model, **YOU MAY NEED TO ADD FILTERING LOGIC TO ```ZeroShot.py```** file that is also present in the ```Benchmarking_Pipeline``` folder. Although most of the code is generalizable across multiple models, sidecases always do exist:

In [ ]:
model_name = "gpt-5"

# Three possible options for mode: huggingface, openai (only for GPT-models), genai-client (only for Gemini-based models):
mode = "openai"

# Specify which folder to store the .yaml files:
results_folder = ""
os.makedirs(results_folder, exist_ok = True)

### Step #4: Run the Pipeline:

The following cell runs the entire pipeline and collects the results.

In [ ]:
latencies = []
input_tokens = []
output_tokens = []

# If it is a GPT Model, only run the results ONCE:
if mode == "openai":

    for i in range(0, 1):
        client = ZeroShot(model_name, mode, eval_df)
        pipeline = KQLBenchmarkPipeline(client)
        pipeline.run()
        pipeline.save_results(f"{results_folder}/{model_name}-zero-shot-{i}.yaml")
        latencies.append((sum(client.times))/(len(client.times)))

        revised_name = model_name

# All other models - run five times:
else:
    for i in range(0, 5):
        client = ZeroShot(model_name, mode, eval_df)
        pipeline = KQLBenchmarkPipeline(client)
        pipeline.run()
        
        if mode == 'huggingface':
            revised_name = re.search(r'\/(.*)', model_name, flags=re.DOTALL).group(1)
        else:
            revised_name = model_name
            
        pipeline.save_results(f"{results_folder}/{revised_name}-zero-shot-{i}.yaml")
        latencies.append((sum(client.times))/(len(client.times)))

        # Take the input/output tokens, and move the model to cpu:
        if mode == 'huggingface':
            input_tokens.append(client.input_tokens)
            output_tokens_local = 0
            for idx, row in client.df.iterrows():
                output_tokens_local = output_tokens_local + len(client.tokenizer(row['Full Response'][0]['generated_text'])["input_ids"])
            
            output_tokens.append(output_tokens_local)
            client.model = client.model.to("cpu")

        if mode == 'genai-client':
            genai_client = genai.Client(api_key = token_config['genai']['token'], http_options=HttpOptions(timeout=3*60*1000))
            
            input_tokens_local = 0
            output_tokens_local = 0
            count = 0
            
            for idx, row in client.df.iterrows():
                input_tokens_local += genai_client.models.count_tokens(model="gemini-2.0-flash", contents = f"You are a programmer using the Kusto Query Language with Microsoft Defender. Generate a KQL query that answers the following request. Return only the KQL code without any explanation. {eval_df.loc[count]['context']}").total_tokens
                output_tokens_local += genai_client.models.count_tokens(model="gemini-2.0-flash", contents = str(row['Full Response'])).total_tokens
                count = count + 1

            input_tokens.append(input_tokens_local)
            output_tokens.append(output_tokens_local)

In [ ]:
num_queries = len(client.df)           # or client.df.shape[0]
print("Processed queries:", num_queries)

In [ ]:
print(len(latencies))

In [ ]:
print(len(client.times))

### Step #5: Merge Results together

The following saves the results into a yaml file. Please note that while we have code that has done extensive regex cleaning, manual revision is still required in order to check for any extraneous characters that might affect the metrics scores.

In [ ]:
import glob, yaml

# Merge any shards you have (adjust the glob if needed)
paths = sorted(glob.glob(f"{results_folder}/{model_name}-zero-shot-*.yaml"))

merged = {"queries": []}
for p in paths:
    with open(p, "r") as f:
        y = yaml.safe_load(f) or {}
        merged["queries"].extend(y.get("queries", []))

with open(f"{results_folder}/{model_name}-zero-shot.yaml", "w") as f:
    yaml.dump(merged, f, sort_keys=False, default_style='|', allow_unicode=True, width=1000)

### Step #6: Latency Output

Prints out the average latency for all queries.

In [ ]:
# Run this cell:
print(f"Avg. Latency: {round((sum(client.times))/(len(client.times)), 3)}")

### Step #7: Cost Analysis:

Prints out the average cost for running all queries. We calculate the costs in different ways:
1. ##### HuggingFace Models:
   - We calculate the number of input and output tokens as the queries are run. Once we calculate the total number of input and output tokens, we save them into list variables and can calculate the total costs from there.
  
2. ##### OpenAI Models:
   - We use the ```tiktoken``` package to calculate the OpenAI model input and output tokens.
  
3. ##### GenAI-Client Models:
    - Due to potential risks of crashing, we calculate tokens AFTER all results have been obtained.
  
In ALL cases, you must update the input token costs and output token costs (per million tokens).

In [ ]:
import math
import tiktoken
import pandas as pd

# === PRICING (USD per 1M tokens) ===
llm_input_cost_per_million  = 1.250
llm_output_cost_per_million = 10.000

# ===== helper: robust encoder getter =====
def _encoding_name_for(model_name: str) -> str:
    m = (model_name or "").lower()
    if any(x in m for x in ["gpt-4o", "gpt-4.1", "4o-mini", "o200k", "gpt-5"]):
        return "o200k_base"
    if any(x in m for x in ["gpt-4", "gpt-3.5", "cl100k", "text-embedding-3"]):
        return "cl100k_base"
    if any(x in m for x in ["text-davinci-003", "code-davinci", "p50k"]):
        return "p50k_base"
    return "cl100k_base"

def get_tiktoken_encoder(model_name: str = None):
    """Return a tiktoken encoding object (not a string)."""
    try:
        if model_name:
            return tiktoken.encoding_for_model(model_name)
    except Exception:
        pass
    # fallback: return encoding by name
    return tiktoken.get_encoding(_encoding_name_for(model_name or ""))

# ====== Safe cost calculation ======
if mode == "openai":
    # choose model name you used for the API call; use None if unknown
    model_used = "gpt-5"
    enc = get_tiktoken_encoder(model_used)

    # prepare prompt template (same as you used)
    prompt_tmpl = (
        "You are a programmer using the Kusto Query Language with Microsoft Defender. "
        "Generate a KQL query that answers the following request. "
        "Return only the KQL code without any explanation. {context}"
    )

    # How many examples to count (do not exceed eval_df length)
    max_examples = 10
    n_examples = min(max_examples, len(eval_df))

    input_tokens = 0
    for i in range(n_examples):
        ctx = ""
        # safe access: .iloc handles integer position
        try:
            ctx = eval_df.iloc[i].get("context", "") if isinstance(eval_df.iloc[i], dict) else eval_df.iloc[i].get("context", "") 
        except Exception:
            # fallback: use .loc with index if eval_df is normal DataFrame
            try:
                ctx = eval_df.loc[eval_df.index[i], "context"]
            except Exception:
                ctx = ""
        if pd.isna(ctx):
            ctx = ""
        prompt = prompt_tmpl.format(context=str(ctx))
        input_tokens += len(enc.encode(prompt))

    # Count output tokens from client.df['Full Response']
    output_tokens = 0
    if hasattr(client, "df") and "Full Response" in client.df.columns:
        for resp in client.df["Full Response"]:
            if isinstance(resp, str) and len(resp):
                output_tokens += len(enc.encode(resp))
    else:
        # fallback: try to find a response-like column
        found = False
        if hasattr(client, "df"):
            for c in client.df.columns:
                if "resp" in c.lower() or "full" in c.lower() or "response" in c.lower():
                    for resp in client.df[c]:
                        if isinstance(resp, str) and len(resp):
                            output_tokens += len(enc.encode(resp))
                    found = True
                    break
        if not found:
            print("Warning: couldn't find client.df['Full Response'] or alternative; output_tokens set to 0")

    cost = (llm_input_cost_per_million * input_tokens) / 1_000_000.0 \
         + (llm_output_cost_per_million * output_tokens) / 1_000_000.0

    print(f"Input tokens:  {input_tokens:,}")
    print(f"Output tokens: {output_tokens:,}")
    print(f"Average Total Cost: ${cost:.3f}")

elif mode in ("huggingface", "genai-client"):
    # Expect you have per-prompt lists of tokens (input_tokens_list, output_tokens_list)
    # If you only have aggregated numbers, adapt accordingly.
    # Example: if you kept lists:
    try:
        input_list = input_tokens  # in your original code this might be a list
        output_list = output_tokens
        if not isinstance(input_list, list) or not isinstance(output_list, list):
            raise TypeError("expected input_tokens and output_tokens to be lists of per-prompt counts")
        if len(input_list) != len(output_list):
            print("Warning: input/output lists differ in length; pairing by min length")
        pairs = list(zip(input_list, output_list))
    except Exception:
        # If you only have totals (int), wrap them so the code still runs
        pairs = [(int(input_tokens), int(output_tokens))]

    total_cost = 0.0
    for itoks, otoks in pairs:
        total_cost += (llm_input_cost_per_million * int(itoks)) / 1_000_000.0 \
                    + (llm_output_cost_per_million * int(otoks)) / 1_000_000.0

    runs = 5  # adjust to the number of runs you averaged over
    avg_cost = total_cost / max(1, runs)
    print(f"Average Total Cost over {runs} runs: ${avg_cost:.3f}")

else:
    raise ValueError(f"Unknown mode: {mode}")


### Step #8: Get Metrics:

The following snippets will obtain all metrics for the results that you have just calculated. Please note that full path **must** be specified for ```file_of_interest``` and ```folder```. If you need help specifying the full path, use the ```!pwd``` command in Jupyter notebook.

In [ ]:
# DO NOT CHANGE THE FOLLOWING VARIABLE:
runner = "../offline_metrics_pipeline/offline-metrics-pipeline/offline_metrics_runner.py"

# CHANGE THE FOLLOWING VARIABLES:

# This should point to where your .yaml is currently stored (entire path must be specified):
file_of_interest = ""

# These should point to where you would like to store your results (entire path must be specified):
folder = ""
results_file = "testing.csv"

In [ ]:
!python {runner} {file_of_interest} {folder} {results_file}